In [2]:
%load_ext autoreload
%autoreload 2
import ObjectiveFunction as of
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import helper_funcs as hf
from math import sqrt
from scipy.stats import uniform, norm


In [3]:
from torch.optim.optimizer import Optimizer, required
import torch
from typing import List, Optional

class SGDMeta(Optimizer):

    def __init__(self,
                 params,
                 height: float = required,
                 width: float = required,
                 lr: float = required,
                 momentum: float = 0
    ):

        if lr is not required and lr < 0.0:
            raise ValueError(f"Invalid learning rate: {lr}")
        if height is not required and height < 0.0:
            raise ValueError(f"Invalid height value: {height}")
        if width is not required and width < 0.0:
            raise ValueError(f"Invalid width value: {width}")
        if momentum < 0.0:
            raise ValueError(f"Invalid momentum value: {momentum}")

        self.height = height
        self.width_denom = -0.5*(1/width)**2
        defaults = dict(height=height, width=width, lr=lr, momentum=momentum)
        super().__init__(params, defaults)
    
    def _metric(self, pred):

        if not self.state:
            return pred
        
        history = self.state['history'][0:-1]
        last_ph = self.state['history'][-1]
        
        Vbias = 0

        for ph in history:
            v = last_ph - ph
            Vbias += torch.exp(self.width_denom * torch.dot(v, v.T))
        
        Vbias = self.height * Vbias

        # update detachment of tensors
        self.state['history'][-1] = self.state['history'][-1].detach().clone()
        
        return pred + Vbias


    @torch.no_grad()
    def step(self, closure=None):

        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure() # learn what a closure is 21/04/2022
        
        # Get the current parameter state

        for group in self.param_groups:
            for p in group['params']:
                grad = p.grad
                if 'history' not in self.state:
                    self.state['history'] = [p]
                else:
                    self.state['history'].append(p)

                if 'momentum_buffer' not in self.state:
                    self.state['momentum_buffer'] = grad.detach().clone()
                else:
                    self.state['momentum_buffer'].mul_(group['momentum']).add_(grad, alpha=1)
                grad = self.state['momentum_buffer']
                p.add_(grad, alpha=-group['lr'])

                r = float(uniform.rvs(loc=0, scale=0.01, size=1)) * torch.norm(grad)
                theta = float(norm.rvs(loc=0, scale=0.1*np.pi, size=1))
                noise = torch.Tensor([r*np.cos(theta), r*np.sin(theta)])
                p.add_(noise, alpha=-group['lr'])

        return loss



In [160]:
m = of.Ackley(start=[1.0, 1.2], bounds=5)
ackley_sgdlevy = of.OptimisationProblem(
    m,
    SGDMeta(m.parameters(), height=0.2, width=0.02, lr=0.01, momentum=0),
    n_epochs = 800
)
losses, params, preds = ackley_sgdlevy.run()
fig1 = ackley_sgdlevy.visualise((-2, 2), (-2, 2), 0.1, render="contour")
fig1.show()

In [162]:
m = of.Ackley(start=[1.0, 1.2], bounds=5)
ackley_sgdlevy = of.OptimisationProblem(
    m,
    torch.optim.Adam(m.parameters(), lr=0.05),
    n_epochs = 800
)
losses, params, preds = ackley_sgdlevy.run()
fig2 = ackley_sgdlevy.visualise((-2, 2), (-2, 2), 0.1, render="contour")
fig2.show()

In [161]:
m = of.Ackley(start=[1.0, 1.2], bounds=5)
ackley_sgdlevy = of.OptimisationProblem(
    m,
    SGDMeta(m.parameters(), height=0.2, width=0.02, lr=0.01, momentum=0.75),
    n_epochs = 800
)
losses, params, preds = ackley_sgdlevy.run()
fig3 = ackley_sgdlevy.visualise((-2, 2), (-2, 2), 0.1, render="contour")
fig3.show()

In [163]:
hf.figures_to_html([fig1,fig2, fig3], 'metadynamics_cherry_pick.html')

In [5]:
# check performance of SGDMeta with Momentum on Alpine
m = of.AlpineN1(start=[2.5, 2], bounds=10)
alpline_META = of.OptimisationProblem(
    m,
    SGDMeta(m.parameters(), height=0.2, width=0.02, lr=0.01, momentum=0.75),
    n_epochs = 1000
)

losses, params, preds = alpline_META.run()
fig3 = alpline_META.visualise((-10, 10), (-10, 10), 0.1, render="contour")
fig3.show()

In [164]:
runs = []
for _ in range(0, 100):

    m = of.Ackley(start=[1.0, 1.2], bounds=5)
    ackley_sgdlevy = of.OptimisationProblem(
        m,
        SGDMeta(m.parameters(), height=0.2, width=0.02, lr=0.01, momentum=0.75),
        n_epochs = 800
    )
    losses, params, preds = ackley_sgdlevy.run()
    runs.append(ackley_sgdlevy)
    if _ % 10 == 0: print(_)

0
10
20
30
40
50
60
70
80
90


In [166]:
paths = [run.params for run in runs]
fig4, ts, emsds = hf.emsd_analysis(paths, tau=20)
alphas = hf.derivative(np.log(ts), np.log(emsds))
fig5 = hf.plot(ts, emsds, 't', 'eMSD', title='metadynamics_momentum75_h20_width2_lr1_emsd')
fig6 = hf.plot(ts, alphas, 't', 'α', title='metadynamics_momentum75_h20_width2_lr1_emsd_alpha')

C:\Users\karls\AppData\Local\Temp/ipykernel_16164/1645313383.py:3: RuntimeWarning:

divide by zero encountered in log



In [167]:
hf.figures_to_html([fig1,fig2, fig3, fig4, fig5, fig6], 'metadynamics_cherry_pick.html')

In [11]:
from torch.optim.optimizer import Optimizer, required
import torch
from typing import List, Optional
from scipy.stats import uniform, norm

class Queue():

    def __init__(self, size):
        self.data = []
        self.size = size
    
    def get(self, index=0):

        if len(self.data) == 0:
            raise IndexError("Cannot get from empty Queue.")

        return self.data[index]

    def enqueue(self, item):

        if len(self.data) == self.size:
            self.deqeue()
        
        self.data.append(item)
    
    def deqeue(self):
        return self.data.pop(0)

    def __repr__(self):
        return str(self.data)

    def __getitem__(self, index):
        return self.data[index]

    def __setitem__(self, index, value):
        self.data[index] = value

class SGDMetaCache(Optimizer):

    def __init__(self,
                 params,
                height: float = required,
                 width: float = required,
                 lr: float = required,
                 cachesize: int = required,
                 momentum: float = 0
    ):

        if lr is not required and lr < 0.0:
            raise ValueError(f"Invalid learning rate: {lr}")
        if momentum < 0.0:
            raise ValueError(f"Invalid momentum value: {momentum}")
        if height is not required and height < 0.0:
            raise ValueError(f"Invalid height value: {height}")
        if width is not required and width < 0.0:
            raise ValueError(f"Invalid width value: {width}")
        if cachesize <= 0:
            raise ValueError(f"Invalid cachesize: {cachesize}")
       
        self.height = height
        self.width_denom = -0.5*(1/width)**2
        defaults = dict(height=height, cachesize=cachesize, width=width, lr=lr, momentum=momentum)
        super().__init__(params, defaults)

    def _metric(self, pred):

        if 'cache' not in self.state:
            return pred
        
        history = self.state['cache'][0:-1]
        last_ph = self.state['cache'][-1]
        
        Vbias = 0

        for ph in history:
            v = last_ph - ph
            Vbias += torch.exp(self.width_denom * torch.dot(v, v.T))
        
        Vbias = self.height * Vbias

        # update detachment of tensors
        self.state['cache'][-1] = self.state['cache'][-1].detach().clone()
        
        return pred + Vbias

    @torch.no_grad()
    def step(self, closure=None):

        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()
        
        # Get the current parameter state

        for group in self.param_groups:
            for p in group['params']:
                grad = p.grad

                if 'momentum_buffer' not in self.state:
                    self.state['momentum_buffer'] = grad.detach().clone()
                else:
                    self.state['momentum_buffer'].mul_(group['momentum']).add_(grad, alpha=1)
                grad = self.state['momentum_buffer']
                p.add_(grad, alpha=-group['lr'])

                if 'cache' not in self.state:
                    self.state['cache'] = Queue(size=group['cachesize'])
                self.state['cache'].enqueue(p)

                if 'grad_store' not in self.state:
                    self.state['grad_store'] = [grad.detach().clone()]
                else:
                    self.state['grad_store'].append(grad.detach().clone())
                
                r = float(uniform.rvs(loc=0, scale=0.01, size=1)) * torch.norm(grad)
                theta = float(norm.rvs(loc=0, scale=0.1*np.pi, size=1))
                noise = torch.Tensor([r*np.cos(theta), r*np.sin(theta)])
                p.add_(noise, alpha=-group['lr'])

                # r = float(uniform.rvs(loc=0, scale=0.01, size=1)) * torch.norm(grad)
                # theta = float(norm.rvs(loc=0, scale=0.1*np.pi, size=1))
                # noise = torch.Tensor([r*np.cos(theta), r*np.sin(theta)])
                # p.add_(noise, alpha=-group['lr'])

        return loss

In [12]:
m = of.AlpineN1(start=[2.5, 2.5], bounds=10)
alpline_sgdlevy2 = of.OptimisationProblem(
    m,
    SGDMetaCache(m.parameters(), height=0.2, width=0.02, lr=0.01, momentum=0.85, cachesize=500),
    n_epochs = 1000
)

losses, params, preds = alpline_sgdlevy2.run()
fig = alpline_sgdlevy2.visualise((-10, 10), (-10, 10), 0.1, render="contour")
fig.show()

In [10]:
"""
Use stored distribution of the angle of updates to indicate direction
"""
# Investigate angular distribution

opt_prob = alpline_sgdlevy2
opt = opt_prob.opt
grad_store = [list(np.array(_)) for _ in opt.state['grad_store']]
thetas = np.arctan2(np.array([a[1] for a in grad_store]), np.array([a[0] for a in grad_store])) * 180 / np.pi

# only works in 2 dimensions for now - need a transformation from cartesian to nsphere coordinates
# https://en.wikipedia.org/wiki/N-sphere#Spherical_coordinates has inverse

fig = go.Figure()
fig.add_trace(
    go.Histogram(x=thetas)
)
fig.show()
